In [1]:
from model import FFNet, tv_model
from learning import Learn, Selector
from dataset import SKDS, TVDS

from torch.optim import Adam
from torch.nn import MSELoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms


In [ ]:
model_params = {'D_in': 128, 
                'H': 512, 
                'D_out': 1, 
                'model_name': 'funnel'}

ds_params = {'make': 'make_regression',
             'sk_params': {'n_samples': 10000,
                           'n_features': 128,
                           'transform': transforms.Compose(
                                               [transforms.Resize(224),
                                                transforms.ToTensor()])}}

opt_params = {'lr': 0.01}
crit_params = {'reduction': 'sum'}
sample_params = {'set_seed': 88,
                 'splits': (.1,.8)}
sched_params = {'factor': .1,
                'patience': 2}


l = Learn(SKDS, FFNet, Selector, Optimizer=Adam, Scheduler=ReduceLROnPlateau, Criterion=MSELoss,
          model_params=model_params, ds_params=ds_params, sample_params=sample_params,
          opt_params=opt_params, sched_params=sched_params, crit_params=crit_params,
          adapt=False, load_model=False, load_embed=False, save_model=False,
          batch_size=256, epochs=10)

In [ ]:
model_params = {'D_in': 784, 
                'H': 1024, 
                'D_out': 10, 
                'model_name': 'funnel'}

ds_params={'embed': None,
           'train_params': {'dataset': 'MNIST',
                            'tv_params':{'root': './data/',
                                         'train': True,
                                         'download': True,
                                         'transform': transforms.Compose(
                                                           [transforms.Resize(224),
                                                            transforms.ToTensor()]),
                                         'target_transform': None}
                            
            'test_params': {'dataset': 'MNIST',
                            'tv_params':{'root': './data/',
                                         'train': False,
                                         'download': True,
                                         'transform': transforms.Compose(
                                                           [transforms.Resize(224),
                                                            transforms.ToTensor()]),
                                         'target_transform': None}
        

opt_params = {'lr': 0.01}
crit_params = {'reduction': 'sum'}
sample_params = {'set_seed': 88,
                 'splits': (.7,.15)}
sched_params = {'factor': .1,
                'patience': 2}

l = Learn([TVDS,TVDS], FFNet, Selector, Optimizer=Adam, Scheduler=ReduceLROnPlateau, Criterion=MSELoss, 
          model_params=model_params, ds_params=ds_params, sample_params=sample_params,
          opt_params=opt_params, sched_params=sched_params, crit_params=crit_params,
          adapt=False, load_model=False, load_embed=False, save_model=False,
          batch_size=256, epochs=10)

In [2]:
model_params = {'model_name': 'resnet18',
                'D_in': 28,
                'D_out': 10,
                'tv_params': {}}

ds_params={'embed': None,
           'ds_params': {'dataset': 'MNIST',
                         'tv_params':{'root': './data/',
                                      'train': True,
                                      'download': True,
                                      'transform': transforms.Compose(
                                                        [transforms.ToTensor()]),
                                      'target_transform': None}}}
           
opt_params = {'lr': 0.01}
crit_params = {'reduction': 'sum'}
sample_params = {'set_seed': 88,
                 'splits': (.7,.15)}
sched_params = {'factor': .1,
                'patience': 10}

l = Learn([TVDS], tv_model, Selector, Optimizer=Adam, Scheduler=ReduceLROnPlateau, Criterion=MSELoss, 
          model_params=model_params, ds_params=ds_params, sample_params=sample_params,
          opt_params=opt_params, sched_params=sched_params, crit_params=crit_params,
          adapt=False, load_model=False, load_embed=False, save_model=False,
          batch_size=16, epochs=10)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[16, 1, 28, 28] to have 3 channels, but got 1 channels instead

In [ ]:
ds_params={'dataset': 'MNIST',
           'embed': None,
           'tv_params': {'root': './data/',
                         'train': True,
                         'download': True,
                         'transform': transforms.Compose(
                                           [transforms.Resize(8),
                                            transforms.ToTensor()]),
                         'target_transform': None}}
           
tvds = TVDS(**ds_params)

In [ ]:
tvds[1]

In [ ]:
tvds[2][0].shape

In [ ]:
model_params = {'model_name': 'resnet18',
                'D_in': 224,
                'D_out': 10,
                'tv_params': {}}

tvmodel = tv_model(**model_params)

In [ ]:
dir(tvmodel)

In [ ]:
type(tvmodel)

In [ ]:
isinstance(tvmodel, torchvision.models.resnet.ResNet)